In [13]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import nltk
nltk.download("gutenberg")
from nltk.corpus import gutenberg
import pandas as pd 
import mlflow
import os
# os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"
# mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
# try:
#     mlflow.set_experiment("lstm_gru")
# except mlflow.exceptions.MlflowException:
#     mlflow.create_experiment("lstm_gru")
#     mlflow.set_experiment("lstm_gru")

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/sarthakagarwal/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [14]:
# data = gutenberg.raw('shakespeare-hamlet.txt')

In [15]:
# with open("data/hamlet.txt", "w") as file:
#     file.write(data)

In [16]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [17]:
with open("data/hamlet.txt", "r") as file:
    text = file.read().lower()

In [18]:
# Index creation for words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
print(total_words)


4818


In [19]:
# input sequence creation
input_sequence = []

for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequence.append(n_gram_sequence)
    

In [20]:
# pad sequence 
max_sequence_length = max(len(i) for i in input_sequence)
input_sequence = np.array(
    pad_sequences(
        input_sequence, 
        maxlen=max_sequence_length,
        padding='pre'
    )
)

In [21]:
# create predictor and labels
import tensorflow as tf 

x, y = input_sequence[:, :-1], input_sequence[:, -1]

In [22]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=0.2,
    random_state=42
)

In [23]:
# train our lstm model on the dataset
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(GRU(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(150))
model.add(Dense(total_words, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


# model=Sequential()
# model.add(Embedding(total_words,100,input_length=max_sequence_length-1))
# model.add(GRU(150,return_sequences=True))
# model.add(Dropout(0.2))
# model.add(GRU(100))
# model.add(Dense(total_words,activation="softmax"))

# # #Compile the model
# model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
# model.summary()


callback = EarlyStopping(monitor='val_loss', patience=20,
                         restore_best_weights=True)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# mlflow.autolog()
# with mlflow.start_run(run_name="lstm"):
history = model.fit(
    x_train,
    y_train,
    epochs=100,
    validation_data=(x_test, y_test),
    verbose=1,
    callbacks=[callback]
)

Epoch 1/100


2025-04-07 14:16:50.734570: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


107/644 ━━━━━━━━━━━━━━━━━━━━ 13s 25ms/step - accuracy: 0.0251 - loss: 7.7950

In [50]:
def predict(model, tokenizer, text, max_sequence_length):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_length:
        token_list = token_list[-max_sequence_length+1:]
    token_list = pad_sequences(
        [token_list],
        maxlen=max_sequence_length-1,
        padding='pre',
    )
    predicted = model.predict(token_list, verbose=0)
    preicted_word_index = np.argmax(predicted, axis = 1)
    for word, index in tokenizer.word_index.items():
        if index == preicted_word_index:
            return word
    
    return None

In [ ]:
predict(
    model,
    tokenizer=tokenizer,
    max_sequence_length=max_sequence_length,
    text="to be or not to be",
)

In [ ]:
model.save("model_gru.h5")
import pickle
with open("tokenizer_gru.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)